# Entrenamiento para identificar la tipología fotográfica
![Tipologías fotograficas](imagenes/tipologias.jpg)

Este cuaderno se basa en la técnica de re entrenamiento o transferencia de aprendizaje y se realizó siguiendo las instrucciones del repositorio [Tensorflow for poets](https://github.com/googlecodelabs/tensorflow-for-poets-2) en la versión adaptada por Tim Sherrat en sus colleción [Glam Workbench] (https://github.com/GLAM-Workbench/image-recognition) Se usa una Red Neuronal Convolucional llamada MobileNet para crear un modelo que permite clasificar imagenes de acuerdo a su tipología fotográfica.

NOTAS.
- Originalmente este cuaderno lo trabajé en el sistema operativo linux por lo que algunos comandos son los de ese ambiente. 
- Las celdas que empiezan con los simbolos ! o $$ se ejecutan fuera del cuaderno jupyter



Se debe descargar el repositorio que contiene los scripts para el re entrenamiento. Las imagenes que se usarán las obtendremos independientemente.

In [ ]:
!git clone https://github.com/googlecodelabs/tensorflow-for-poets-2 

Verificamos que el cuaderno tenga acceso a la version 1.11.0 del modulo Tensorflow. Esta es una versión muy vieja por lo que hay que ser cuidadosos al crear en environment de Python para obtener la versión que necesitamos 

In [ ]:
import tensorflow as tf

print(tf.__version__)

Accedemos al directorio que se acaba de descargar

In [ ]:
cd tensorflow-for-poets-2

In [ ]:
!pwd #Muestra el directorio actual

## Categorias de entrenamiento

Dentro de la carpeta tf_files bebemos copiar nuestras fotografías, para cada categorìa de clasificación usaremos un folder diferente y todos ellos deben estar en la carpeta training

In [ ]:
ls tf_files

## Re entrenamiento

Se ejecuta el archivo retrain.py de la carpeta scripts que agrega una nueva capa final a la red y realiza el re entrenamiento. 
![convolución](imagenes/convolucion.jpg)

Pero antes de hacerlo se definen variables para el tamaño de imagen (128,160,192, or 224px) y la arquitectura y tamaño de la red (1.0, 0.75, 0.50, or 0.25.). También se pasan otros parametros con los simbolos -- Como son varias lineas, se usan saltos de linea con \ para mayor claridad.

Si se omite la variable para la arquitectura de la red neuronal se usara por default la red Inception V3, sin embargo en este cuaderno se usa la red Mobilenet que es más pequeña y eficiente, aunque tambien menos precisa. Ambas redes han sido entrenadas con el conjunto de datos Imagenet.

In [ ]:
%%bash
IMAGE_SIZE=224
ARCHITECTURE="mobilenet_0.50_${IMAGE_SIZE}"

python -m scripts.retrain \
  --bottleneck_dir=tf_files/bottlenecks \
  --how_many_training_steps=500 \
  --model_dir=tf_files/models/ \
  --summaries_dir=tf_files/training_summaries/"${ARCHITECTURE}" \
  --output_graph=tf_files/retrained_graph.pb \
  --output_labels=tf_files/retrained_labels.txt \
  --architecture="${ARCHITECTURE}" \
  --image_dir=tf_files/training

Al final el script produce dos archivos nuevos

    - tf_files/retrained_graph.pb, la nueva red que hemos re entrenado
    - tf_files/retrained_labels.txt, el archivo de texto con las categorias en las que dividimos las fotos

## Prueba de clasificación 

Primero con una fotografía al azar del conjunto de entrenamiento tf_files/training

In [ ]:
# Make a list of all the train images
import os
import random
from IPython.display import display, HTML
imgs = []
data_dir = 'tf_files/training'
for img_dir in [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]:
    for img in [i for i in os.listdir(os.path.join(data_dir, img_dir)) if i[-4:] == '.jpg']:
        imgs.append(os.path.join(data_dir, img_dir, img))    

In [ ]:
# Choose one image at random
img = random.sample(imgs, 1)[0]
display(HTML('<img src="tensorflow-for-poets-2/{0}" width="50%" height="50%"><br>{0}'.format(img)))
print(img)

Se ejecuta el archivo llamado label_image, ubicado en la carpeta scripts, 
usando el nuevo modelo y el nuevo archivo de etiquetas

In [ ]:
!python -m scripts.label_image --graph=tf_files/retrained_graph.pb --labels=tf_files/retrained_labels.txt --image=$img

Luego con una fotografía que el sistema no conoce de la carpeta tf_files/testing

In [ ]:
# Make a list of all the test images
import os
import random
from IPython.display import display, HTML
imgs = []
data_dir = 'tf_files/testing'
for img_dir in [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]:
    for img in [i for i in os.listdir(os.path.join(data_dir, img_dir)) if i[-4:] == '.jpg']:
        imgs.append(os.path.join(data_dir, img_dir, img))    

In [ ]:
# Choose one image at random
img = random.sample(imgs, 1)[0]
display(HTML('<img src="tensorflow-for-poets-2/{0}" width="50%" height="50%"><br>{0}'.format(img)))
print(img)

In [ ]:
!python -m scripts.label_image --graph=tf_files/retrained_graph.pb --labels=tf_files/retrained_labels.txt --image=$img

## Imagen de nuestra elección
Solo debemos copiar la ruta de carpetas y el nombre del archivo que queremos clasificar

In [ ]:
img= "tf_files/testing/Impresion/STUDY152.jpg"

In [ ]:
display(HTML('<img src="tensorflow-for-poets-2/{0}" width="50%" height="50%"><br>{0}'.format(img)))
print(img)

In [ ]:
!python -m scripts.label_image --graph=tf_files/retrained_graph.pb --labels=tf_files/retrained_labels.txt --image=$img